In [1]:
import networkx as nx
import itertools

def chinese_postman_problem(G):
    """
    Solves the Chinese Postman Problem for an undirected graph G.
    Returns the augmented graph with an Eulerian circuit.
    """
    # Step 1: Check if the graph is connected
    if not nx.is_connected(G):
        raise nx.NetworkXError("Graph is not connected.")

    # Step 2: Find all vertices with odd degree
    odd_degree_nodes = [v for v, d in G.degree() if d % 2 == 1]

    # If there are no odd-degree nodes, the graph is Eulerian
    if len(odd_degree_nodes) == 0:
        euler_circuit = list(nx.eulerian_circuit(G))
        return euler_circuit

    # Step 3: Find all pairs of odd-degree vertices and calculate shortest paths between them
    odd_pairs = list(itertools.combinations(odd_degree_nodes, 2))
    distances = {}
    paths = {}
    for u, v in odd_pairs:
        length, path = nx.single_source_dijkstra(G, u, v)
        distances[(u, v)] = length
        paths[(u, v)] = path

    # Step 4: Create a complete graph of odd-degree vertices with edge weights as distances
    complete_graph = nx.Graph()
    complete_graph.add_nodes_from(odd_degree_nodes)
    for u, v in odd_pairs:
        complete_graph.add_edge(u, v, weight=distances[(u, v)])

    # Step 5: Find the minimum weight matching in the complete graph
    min_weight_matching = nx.algorithms.matching.min_weight_matching(complete_graph, maxcardinality=True)

    # Step 6: Augment the original graph by adding duplicate edges along the shortest paths
    augmented_graph = G.copy()
    for u, v in min_weight_matching:
        path = paths[(u, v)]
        edges_in_path = list(zip(path[:-1], path[1:]))
        for edge in edges_in_path:
            u_edge, v_edge = edge
            if augmented_graph.has_edge(u_edge, v_edge):
                # Increment the 'duplex' attribute to indicate duplicated edge
                if 'duplex' in augmented_graph[u_edge][v_edge]:
                    augmented_graph[u_edge][v_edge]['duplex'] += 1
                else:
                    augmented_graph[u_edge][v_edge]['duplex'] = 2  # Original edge plus one duplicate
            else:
                # Add the edge if it doesn't exist
                augmented_graph.add_edge(u_edge, v_edge, duplex=1)

    # Step 7: Find an Eulerian circuit in the augmented graph
    euler_circuit = list(nx.eulerian_circuit(augmented_graph))
    return euler_circuit

# Example Usage
if __name__ == "__main__":
    # Create a sample graph
    G = nx.Graph()

    # Add edges (example graph)
    edges = [
        (0, 1, 1),
        (1, 2, 1),
        (2, 3, 1),
        (3, 0, 1),
        (0, 2, 1),
        (1, 3, 1),
        (3, 4, 1),
        (4, 5, 1),
        (5, 2, 1)
    ]
    G.add_weighted_edges_from(edges)

    # Solve the Chinese Postman Problem
    circuit = chinese_postman_problem(G)
    print("Eulerian Circuit:")
    for edge in circuit:
        print(edge)


TypeError: min_weight_matching() got an unexpected keyword argument 'maxcardinality'

In [2]:
import networkx as nx
print(nx.__version__)


3.4.2


In [3]:
import networkx as nx
import itertools

def chinese_postman_problem(G):
    if not nx.is_connected(G):
        raise nx.NetworkXError("Graph is not connected.")

    odd_degree_nodes = [v for v, d in G.degree() if d % 2 == 1]

    if len(odd_degree_nodes) == 0:
        euler_circuit = list(nx.eulerian_circuit(G))
        return euler_circuit

    # Calculate shortest paths between all pairs of odd-degree nodes
    odd_pairs = list(itertools.combinations(odd_degree_nodes, 2))
    distances = {}
    paths = {}
    for u, v in odd_pairs:
        length, path = nx.single_source_dijkstra(G, u, v)
        distances[(u, v)] = length
        paths[(u, v)] = path

    # Create a complete graph of odd-degree vertices with inverted weights
    complete_graph = nx.Graph()
    complete_graph.add_nodes_from(odd_degree_nodes)
    for u, v in odd_pairs:
        weight = distances[(u, v)]
        complete_graph.add_edge(u, v, weight=-weight)  # Invert weights

    # Find the maximum weight matching (which corresponds to the minimum original weights)
    min_weight_matching = nx.algorithms.matching.max_weight_matching(complete_graph, maxcardinality=True)

    # Augment the original graph
    augmented_graph = G.copy()
    for u, v in min_weight_matching:
        # Ensure the order of nodes matches the paths dictionary keys
        u, v = min(u, v), max(u, v)
        path = paths[(u, v)]
        edges_in_path = list(zip(path[:-1], path[1:]))
        for edge in edges_in_path:
            u_edge, v_edge = edge
            if augmented_graph.has_edge(u_edge, v_edge):
                if 'duplex' in augmented_graph[u_edge][v_edge]:
                    augmented_graph[u_edge][v_edge]['duplex'] += 1
                else:
                    augmented_graph[u_edge][v_edge]['duplex'] = 2
            else:
                augmented_graph.add_edge(u_edge, v_edge, duplex=1)

    euler_circuit = list(nx.eulerian_circuit(augmented_graph))
    return euler_circuit

# Example Usage
if __name__ == "__main__":
    # Create a sample graph
    G = nx.Graph()

    # Add edges (example graph)
    edges = [
        (0, 1, 1),
        (1, 2, 1),
        (2, 3, 1),
        (3, 0, 1),
        (0, 2, 1),
        (1, 3, 1),
        (3, 4, 1),
        (4, 5, 1),
        (5, 2, 1)
    ]
    G.add_weighted_edges_from(edges)

    # Solve the Chinese Postman Problem
    circuit = chinese_postman_problem(G)
    print("Eulerian Circuit:")
    for edge in circuit:
        print(edge)


NetworkXError: G is not Eulerian.